# Chapter 9: Stratified Randomization

## Libraries and data

### Libraries

In [4]:
# Common packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific packages
import random # For functions sample() and shuffle()
# To rescale numeric variables
from sklearn.preprocessing import MinMaxScaler
# To one-hot encode cat. variables
from sklearn.preprocessing import OneHotEncoder

### Data

In [5]:
##### Loading the data #####
hist_data_df = pd.read_csv('chap9-historical_data.csv')
exp_data_df = pd.read_csv('chap9-experimental_data.csv')

### Minor data formatting

# Reformating categorical and id variables
hist_data_df['tier'] = pd.Categorical(hist_data_df.tier, categories=[3,2,1], ordered = True)
hist_data_df['ID'] = hist_data_df.ID.astype(str)
exp_data_df['tier'] = pd.Categorical(exp_data_df.tier, categories=[3,2,1], ordered = True)
exp_data_df['ID'] = exp_data_df.ID.astype(str)

hist_data_df.head()

,ID,period,month,sq_ft,tier,avg_review,BPday
0,1,1,2,821.675486,2,9.393427,31.388994
1,2,1,2,745.750743,3,7.392167,47.832222
2,3,1,2,889.114465,3,5.623003,51.075101
3,4,1,2,859.598058,3,6.214651,23.721855
4,5,1,2,963.561800,3,8.096271,21.679488


## Determining random assignment and sample size/power

### Standard randomization

In [6]:
# Function for assignment completely at random with 3 experimental groups
def no_strat_assgnt_fun(dat_df, Nexp):
    temp_df = pd.DataFrame({'ID': hist_data_df.ID.unique()})
    temp_df = temp_df.sample(Nexp)
    grp_lst = ['ctrl', 'treat1', 'treat2'] * int(Nexp / 3)
    random.shuffle(grp_lst)
    temp_df['grp'] = grp_lst 
    return temp_df
no_strat_assgnt = no_strat_assgnt_fun(hist_data_df, Nexp = 999)
no_strat_assgnt.head()

,ID,grp
2911,2912,treat2
4057,4058,treat1
3162,3163,ctrl
2050,2051,treat1
869,870,treat1


In [7]:
# Extension of the previous function for any number K
def no_strat_assgnt_K_fun(dat_df, Nexp, K):
    temp_df = pd.DataFrame({'ID': hist_data_df.ID.unique()})
    temp_df = temp_df.sample(Nexp)
    grp_lst = list(range(K)) * int(Nexp / K)
    random.shuffle(grp_lst)
    temp_df['grp'] = grp_lst 
    return temp_df
no_strat_assgnt = no_strat_assgnt_K_fun(hist_data_df, Nexp = 2000, K = 4)
no_strat_assgnt.head()

,ID,grp
4273,4274,3
4204,4205,1
3374,3375,3
1200,1201,1
2346,2347,0


### Stratified randomization

In [8]:
### Function to prep the data
def strat_prep_fun(dat_df):
    # Making a copy of the input data to avoid side effects
    temp_df = dat_df.copy()
    
    # Extracting property-level variables
    temp_df = temp_df.groupby(['ID', 'tier']).agg(
        sq_ft = ('sq_ft', 'mean'), 
        avg_review = ('avg_review', 'mean'),
        BPday = ('BPday', 'mean'))
    temp_df = temp_df.dropna().reset_index()
    
    num_df = temp_df.copy().loc[:,temp_df.dtypes=='float64'] #Numeric vars 
    cat_df = temp_df.copy().loc[:,temp_df.dtypes=='category'] #Categorical vars

    #Normalizing all numeric variables to [0,1]
    scaler = MinMaxScaler()
    scaler.fit(num_df)
    num_np = scaler.transform(num_df)
    
    #One-hot encoding all categorical variables
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(cat_df)
    cat_np = enc.transform(cat_df).toarray()
    
    #Binding arrays
    data_np = np.concatenate((num_np, cat_np), axis=1)
    del num_df, num_np, cat_df, cat_np, enc, scaler
    return data_np

prepped_data_np = strat_prep_fun(hist_data_df)
prepped_data_np[1:5,:]

/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(


array([[0.75642971, 1.        , 0.73567365, 0.        , 1.        ,
        0.        ],
       [0.44977884, 0.40471599, 0.41839266, 1.        , 0.        ,
        0.        ],
       [0.6341548 , 0.87894388, 0.56860979, 0.        , 0.        ,
        1.        ],
       [0.51756772, 0.77629058, 0.67240189, 1.        , 0.        ,
        0.        ]])

In [9]:
def stratified_assgnt_fun(dat_df, K):
    
    #Sampling down to a multiple of our number of groups
    remainder = len(dat_df) % K
    if remainder != 0:
        dat_df = dat_df.sample(len(dat_df) - remainder)
      
    dat_ID = dat_df.ID.astype(str).tolist() # Extract ID for later join

    match_len = K - 1 # Number of matches we want to find
    match_idx = match_len - 1 # Accounting for 0-indexing
    
    data_np = strat_prep_fun(dat_df)
    N = len(data_np)
    
    #Calculate distance matrix
    from scipy.spatial import distance_matrix
    d_mat = distance_matrix(data_np, data_np)
    np.fill_diagonal(d_mat,N+1)
    # Set up variables
    available = [i for i in range(N)]
    available_temp = available.copy()
    matches_lst = []
    lim = int(N/match_len)
    
    closest = np.argpartition(d_mat, kth=match_idx,axis=1)
    
    for n in available:
        #print("n = ", n)
        if len(matches_lst) == lim: break
        if n in available_temp:
            for match_lim in range(match_idx,N-1):
                #print("match_lim = ", match_lim)
                possible_matches = closest[n,:match_lim].tolist()
                matches = list(set(available_temp) & set(possible_matches))
                #print("len(matches) = ",  len(matches))
                if len(matches) == match_len:
                    matches.append(n)
                    matches_lst.append(matches)
                    available_temp = [m for m in available_temp if m not in matches]
                    break
                else:
                    closest[n,:] = np.argpartition(d_mat[n,:], kth=match_lim)
                    
    #Assigning experimental groups to the matched sets
    exp_grps = np.array(list(range(K))*(int(N/K))).reshape((int(N/K),K))
    exp_grps = exp_grps.tolist()
    for j in exp_grps: 
        np.random.shuffle(j)
    #flattening the two lists
    import itertools
    exp_grps = list(itertools.chain(*exp_grps))
    matches_lst2 = list(itertools.chain(*matches_lst))
    exp_grps2 = [x for _,x in sorted(zip(matches_lst2,exp_grps))]
    
    assgnt_df = pd.DataFrame(exp_grps2, columns=['grp'])
    assgnt_df.grp = assgnt_df.grp.astype(str)
    assgnt_df.grp.loc[assgnt_df.grp == '0'] = 'ctrl'
    assgnt_df.grp.loc[assgnt_df.grp == '1'] = 'treat1'
    assgnt_df.grp.loc[assgnt_df.grp == '2'] = 'treat2'
    
    
    assgnt_df['ID'] = dat_ID
    dat_df = dat_df.merge(assgnt_df, on='ID', how='inner')
    return dat_df

#Sampling a random monthly period
per = random.sample(range(35), 1)[0] + 1
sample_df = hist_data_df.loc[hist_data_df.period == per].sample(5000)
stratified_data_df = stratified_assgnt_fun(sample_df, K=3)
stratified_data_df.head()

/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(


,ID,period,month,sq_ft,tier,avg_review,BPday,grp
0,1472,22,11,699.515663,2,8.120498,44.445122,treat2
1,3712,22,11,629.615685,3,6.590182,36.298660,ctrl
2,490,22,11,733.343112,2,7.837188,38.538355,ctrl
3,3231,22,11,885.843546,1,9.099632,27.268385,treat2
4,1392,22,11,596.921965,1,6.699456,57.907038,ctrl


In [10]:
def assgnt_comparison_fun(strat_dat_df, varnm):
    
    strat_dat_df = stratified_data_df.copy()
    K = 3
    strat_dat_df.rename(columns = {'grp':'strat_grp'}, inplace=True)
    strat_dat_df['assgnt'] = np.random.uniform(0,1,len(strat_dat_df))
    strat_dat_df['grp'] = -1 # initializing the “grp” variable
    for i in range(K):
        strat_dat_df.loc[strat_dat_df['assgnt'].between(i/K, (i+1)/K, inclusive=True), 
               'grp'] = i
    del(strat_dat_df['assgnt'])
    strat_dat_df.rename(columns = {'grp':'no_strat_grp'}, inplace=True)
    
    strat_sd = round(float(strat_dat_df.groupby('strat_grp').agg(var = (varnm, 'mean')).std()), 4)
    print("the s.d. between grps for", varnm, "is", strat_sd, 
          " for stratified assignment\n")
    no_strat_sd = round(float(strat_dat_df.groupby('no_strat_grp').agg(var = (varnm, 'mean')).std()),4)
    print("the s.d. between grps for", varnm, "is", no_strat_sd, 
          "for non-stratified assignment\n") 

assgnt_comparison_fun(stratified_data_df, 'avg_review')
assgnt_comparison_fun(stratified_data_df, 'sq_ft')
assgnt_comparison_fun(stratified_data_df, 'BPday')

ValueError: Inclusive has to be either string of 'both','left', 'right', or 'neither'.

### Power analysis with Bootstrap simulations

In [11]:
# Metric function for free cleaning (treatment 1)
def treat1_metric_fun(dat_df):
    model = ols("BPday~sq_ft+tier+avg_review+grp", data=dat_df)
    res = model.fit(disp=0)
    coeff = res.params['grp[T.treat1]']
    return coeff
    
# Metric function for minimum booking duration (treatment 2)
def treat2_metric_fun(dat_df):
    model = ols("BPday~sq_ft+tier+avg_review+grp", data=dat_df)
    res = model.fit(disp=0)
    coeff = res.params['grp[T.treat2]']
    return coeff

def boot_CI_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
  #Setting sample size
  N = len(dat_df)
  coeffs = []
  
  for i in range(B):
      sim_data_df = dat_df.sample(n=N, replace = True)
      coeff = metric_fun(sim_data_df)
      coeffs.append(coeff)
  
  coeffs.sort()
  start_idx = round(B * (1 - conf_level) / 2)
  end_idx = - round(B * (1 - conf_level) / 2)
  
  confint = [coeffs[start_idx], coeffs[end_idx]]  
  
  return confint

def decision_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, B = B, conf_level = conf_level)
    decision = 1 if boot_CI[0] > 0  else 0
    return decision 

In [12]:
### Function for single experiment

def single_sim_fun(dat_df, metric_fun, Nexp, eff_size, B = 100, 
                   conf_level = 0.9):
    
    #Filter the data down to a random month
    per = random.sample(range(35), 1)[0] + 1
    dat_df = dat_df.loc[dat_df.period == per]
    dat_df = dat_df.sample(n=Nexp)
    
    #Prepare the stratified assignment for a random sample of desired size 
    sample_df = dat_df.sample(Nexp)
    sim_data_df = stratified_assgnt_fun(sample_df, K = 3)
    
    #Add target effect size
    sim_data_df.BPday = np.where(sim_data_df.grp == 'treat2', 
                                 sim_data_df.BPday + eff_size, sim_data_df.BPday)
    
    #Calculate the decision (we want it to be 1)
    decision = decision_fun(sim_data_df, metric_fun, B = B, 
                            conf_level = conf_level)
    return decision
    
single_sim_fun(hist_data_df, treat2_metric_fun, Nexp=99, eff_size=2)

/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(


0

In [13]:
### Functions for simulations at scale
#Standard function
def power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100, 
                  conf_level = 0.9):
    power_lst = []
    for i in range(Nsim):
        power_lst.append(single_sim_fun(dat_df, metric_fun = metric_fun, 
                                        Nexp = Nexp, eff_size = eff_size, 
                                        B = B, conf_level = conf_level))
    power = np.mean(power_lst)
    return power  
power = power_sim_fun(hist_data_df, treat2_metric_fun, Nexp = 1500, eff_size = 2, 
                      Nsim = 100, B = 100, conf_level = 0.9)

/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(
/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(
/tmp/ipykernel_136569/3243074210.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df = temp_df.groupby(['ID', 'tier']).agg(
/tmp/ipykernel_136569/324307

In [14]:
power

0.64

## Analyzing and interpreting experimental results

In [ ]:
#Linear regression
exp_data_reg_df = exp_data_df.copy()
exp_data_reg_df.BPday = np.where((exp_data_reg_df.compliant == 1) & \
                                 (exp_data_reg_df.grp == 'treat2'), 
                                 exp_data_reg_df.BPday -10, 
                                 exp_data_reg_df.BPday) 
print(ols("BPday~sq_ft+tier+avg_review+grp", 
          data=exp_data_reg_df).fit(disp=0).summary())


                            OLS Regression Results                            
Dep. Variable:                  BPday   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     12.26
Date:                Sun, 16 Apr 2023   Prob (F-statistic):           1.68e-13
Time:                        16:52:10   Log-Likelihood:                -6087.8
No. Observations:                1500   AIC:                         1.219e+04
Df Residuals:                    1493   BIC:                         1.223e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        19.2328      3.574      5.382

In [ ]:
print(boot_CI_fun(exp_data_reg_df, treat1_metric_fun))
print(boot_CI_fun(exp_data_reg_df, treat2_metric_fun))

#T-test of means for treatment 1
from statsmodels.stats.weightstats import ttest_ind
test = ttest_ind(exp_data_df[exp_data_df.grp == 'ctrl']['BPday'], 
                 exp_data_df[exp_data_df.grp == 'treat1']['BPday'], 
                 alternative = 'smaller')
test

[-0.4330179102107419, 2.658552318474359]
[-1.194658891588505, 1.7030102814460566]


(-0.9619664409967308, 0.16814973627943702, 998.0)

In [ ]:
#Measuring the compliance rate
exp_data_reg_df.groupby('grp').agg(compliance_rate = ('compliant', 'mean'))

,compliance_rate
grp,
ctrl,1.000
treat1,0.238
treat2,0.166
